# Leitura dos dados

In [2]:
import time

In [3]:
%%time
#============================================================================================#
#                                    IMPORTA BIBLIOTECAS
#============================================================================================#
import pandas as pd
import glob
import numpy as np

#============================================================================================#
#                                    DADOS DE 2003-2023
#============================================================================================#
# cria uma tabela (conhecida também com o nome em inglês de "dataframe") vazia
df_2003_a_2023 = pd.DataFrame()

# lista os arquivos mensais do ano atual
files = sorted(glob.glob('output/01_focos_raw/focos_br_ref_*')) # focos_br_ref_2004.zip

# loop de cada arquivo da lista files
for file in files:

    # leitura da tabela
    df0 = pd.read_csv(file, compression='zip')

    # junta a tabela que foi lida com a anterior
    df_2003_a_2023 = pd.concat([df_2003_a_2023, df0], ignore_index=True)

# remove colunas
df_2003_a_2023.drop(['id_bdq','foco_id','pais'], axis=1, inplace=True)

# renomeia coluna
df_2003_a_2023.rename(columns={'data_pas': 'data'}, inplace=True)

# reposiciona as colunas
df_2003_a_2023 = df_2003_a_2023[['data','lat','lon','municipio','estado','bioma']]

#============================================================================================#
#                                    DADOS DE 2024
#============================================================================================#
# cria uma tabela
df_2024 = pd.DataFrame()

# lista os arquivos mensais do ano atual
files = sorted(glob.glob('output/01_focos_raw/focos_mensal_br_*csv')) # focos_mensal_br_202401.csv

# loop de cada arquivo da lista files
for file in files:

    # leitura da tabela
    df0 = pd.read_csv(file, usecols=['lat', 'lon', 'data_hora_gmt', 'satelite', 'municipio', 'estado', 'bioma'])

    # junta a tabela que foi lida com a anterior
    df_2024 = pd.concat([df_2024, df0], ignore_index=True)

# seleciona apenas para MG e para o satélite de referência AQUA_M-T
df_2024 = df_2024[ df_2024['satelite']=='AQUA_M-T' ]

# remove colunas
df_2024.drop(['satelite'], axis=1, inplace=True)

# renomeia coluna
df_2024.rename(columns={'data_hora_gmt': 'data'}, inplace=True)

# reposiciona as colunas
df_2024 = df_2024[['data','lat','lon','municipio','estado','bioma']]

#============================================================================================#
#                         JUNTA OS ARQUIVOS 2003-2023 E 2024
#============================================================================================#
# junta os dataframes
df_total = pd.concat([df_2003_a_2023, df_2024], ignore_index=True)

# transforma a coluna "datahora" para o formato "datetime"
df_total['data'] = pd.to_datetime(df_total['data'])

# seta a coluna "datahora" como o índice da tabela
df_total.set_index('data', inplace=True)

# ordena a tabela pelo índice
df_total.sort_index(inplace=True)

# mostra o dataframe final
df_total

CPU times: total: 38.2 s
Wall time: 42.9 s


,lat,lon,municipio,estado,bioma
data,,,,,
2003-01-01 16:04:00,-14.82900,-40.13300,NOVA CANAÃ,BAHIA,Mata Atlântica
2003-01-01 16:04:00,-15.62100,-41.66500,BERIZAL,MINAS GERAIS,Mata Atlântica
2003-01-01 16:04:00,-15.27600,-41.51400,TREMEDAL,BAHIA,Mata Atlântica
2003-01-01 16:05:00,-10.11100,-36.50500,IGREJA NOVA,ALAGOAS,Caatinga
2003-01-01 16:05:00,-11.88600,-37.81500,ESPLANADA,BAHIA,Mata Atlântica
...,...,...,...,...,...
2024-10-07 18:47:00,-8.71365,-66.89462,LÁBREA,AMAZONAS,Amazônia
2024-10-07 18:47:00,-8.71499,-66.90406,LÁBREA,AMAZONAS,Amazônia
2024-10-07 18:47:00,-7.94438,-72.48077,CRUZEIRO DO SUL,ACRE,Amazônia


# Processamento

## Função

In [7]:
# Função que calcula o índice i e j da localização do foco
def index(longitudes_matriz, latitudes_matriz, lon_foco, lat_foco):

    ''' Função para calcular o índice (i e j) do pixel de uma matriz que o relâmpago pertence

    Parâmetros:
               longitudes_matriz (array): array de uma dimensão das longitudes da matriz em graus
               latitudes_matriz (array): array de uma dimensão das latitudes da matriz em graus
               lon_raio (float): valor da longitude do foco em graus
               lat_raio (float): valor da latitude do foco em graus

    Retorna:
            indice_lat_raio (float): índice da latitude (ou seja, da linha) do pixel da matriz que o foco pertence
            indice_lon_raio (float): índice da longitude (ou seja, da coluna) do pixel da matriz que o foco pertence
    '''

    # calcula a diferença entre as lats/lons da matriz e a latitude/longitude do foco
    distancia_lon = (longitudes_matriz - lon_foco)**2
    distancia_lat = (latitudes_matriz - lat_foco)**2

    # índice da longitude e latitude do foco
    indice_lon_foco = np.nonzero(distancia_lon == np.min(distancia_lon))
    indice_lat_foco  = np.nonzero(distancia_lat == np.min(distancia_lat))

    # retorna os valores dos índices calculados
    return indice_lat_foco, indice_lon_foco

## Acumula na grade e salva arquivo netcdf do total de focos por ano

In [8]:
# mostra o dataframe final
df_total.head()

,lat,lon,municipio,estado,bioma
data,,,,,
2003-01-01 16:04:00,-14.829,-40.133,NOVA CANAÃ,BAHIA,Mata Atlântica
2003-01-01 16:04:00,-15.621,-41.665,BERIZAL,MINAS GERAIS,Mata Atlântica
2003-01-01 16:04:00,-15.276,-41.514,TREMEDAL,BAHIA,Mata Atlântica
2003-01-01 16:05:00,-10.111,-36.505,IGREJA NOVA,ALAGOAS,Caatinga
2003-01-01 16:05:00,-11.886,-37.815,ESPLANADA,BAHIA,Mata Atlântica


In [10]:
%%time
# importa bibliotecas
import xarray as xr

# Limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# Espaçamento da grade
delta = 20/100.   # grade com 20 km de resolução espacial

# Montando a grade
lons = np.arange(lonmin, lonmax, delta)
lats = np.arange(latmax, latmin, -delta)

# Quantidade de pontos para longitude e latitude
nlon = len(lons)
nlat = len(lats)

# Loop dos anos
for ano in np.arange(2003,2025):

    print('Processando ===>>>', ano)

    # seleciona o mês
    df_selec = df_total.loc[f'{str(ano)}']

    # gera matriz de raios
    focos_lon, focos_lat = df_selec['lon'].values, df_selec['lat'].values

    # interpolando para ponto de grade
    focos = np.zeros((nlat, nlon))

    # loop em cada longitude e latitude da lista
    for lonfoco, latfoco in zip(focos_lon, focos_lat):

        # função que extrai a qual pixel aquele relâmpago pertence
        lin, col = index(lons, lats, lonfoco, latfoco)

        # soma os relâmpagos por pixel
        focos[lin,col]+=1

    # gera arquivo netcdf
    data_vars = {'focos':(('lat', 'lon'), focos, {'units': 'ocorrências/400km²', 'long_name':'Focos de Calor'})}
    coords = {'lat': lats, 'lon': lons, 'time': pd.to_datetime(f'{str(ano)}-12')}
    ds = xr.Dataset(data_vars=data_vars, coords=coords)
    ds.to_netcdf(f'output/03_netcdf_focos_por_ano/focos_anual_brasil_AQUA_{ano}.nc')

    print('Máximo por pixel', np.max(focos), '\n', 'Soma total', np.sum(focos), '\n')

Processando ===>>> 2003
Máximo do Brasil 413.0 
 Máximo do Estado 341237.0 

Processando ===>>> 2004
Máximo do Brasil 460.0 
 Máximo do Estado 380445.0 

Processando ===>>> 2005
Máximo do Brasil 469.0 
 Máximo do Estado 362563.0 

Processando ===>>> 2006
Máximo do Brasil 316.0 
 Máximo do Estado 249179.0 

Processando ===>>> 2007
Máximo do Brasil 456.0 
 Máximo do Estado 393915.0 

Processando ===>>> 2008
Máximo do Brasil 249.0 
 Máximo do Estado 211933.0 

Processando ===>>> 2009
Máximo do Brasil 325.0 
 Máximo do Estado 155102.0 

Processando ===>>> 2010
Máximo do Brasil 338.0 
 Máximo do Estado 319383.0 

Processando ===>>> 2011
Máximo do Brasil 194.0 
 Máximo do Estado 158099.0 

Processando ===>>> 2012
Máximo do Brasil 430.0 
 Máximo do Estado 217234.0 

Processando ===>>> 2013
Máximo do Brasil 168.0 
 Máximo do Estado 128145.0 

Processando ===>>> 2014
Máximo do Brasil 301.0 
 Máximo do Estado 175892.0 

Processando ===>>> 2015
Máximo do Brasil 380.0 
 Máximo do Estado 216778.0 
